In [23]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# import wordcloud
import matplotlib.pyplot as plt

from nltk.corpus import stopwords, opinion_lexicon
from nltk.stem import WordNetLemmatizer

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from tqdm import tqdm
tqdm.pandas()

import os
for dirname, _, filenames in os.walk('aalto-snlp-course-competition-2024'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

aalto-snlp-course-competition-2024/dev_2024.csv
aalto-snlp-course-competition-2024/train_2024.csv
aalto-snlp-course-competition-2024/sample_submission_2024.csv
aalto-snlp-course-competition-2024/test_2024.csv


In [24]:
import csv
df_train = pd.read_csv('./aalto-snlp-course-competition-2024/train_2024.csv',quoting=csv.QUOTE_NONE)
df_dev = pd.read_csv('./aalto-snlp-course-competition-2024/dev_2024.csv',quoting=csv.QUOTE_NONE)
df_test = pd.read_csv('./aalto-snlp-course-competition-2024/test_2024.csv',sep=',',encoding='utf-8',quoting=csv.QUOTE_NONE)

In [25]:
df_train['label'].value_counts()

label
0    62633
1    36367
Name: count, dtype: int64

In [26]:
# Special thanks to https://www.kaggle.com/tanulsingh077 for this function
import re
import string
lemmatizer = WordNetLemmatizer()
def clean_text(text, set_lower=True):
    '''Make text lowercase, remove text in square brackets,remove links, remove html tags,remove punctuation
    and remove numbers.'''
    new_text = text
    if set_lower:
        new_text = new_text.lower()
    # remove text in square brackets
    new_text = re.sub('\[.*?\]', '', new_text)
    # remove links
    new_text = re.sub('https?://\S+|www\.\S+', '', new_text)
    # remove html tags
    new_text = re.sub('<.*?>', '', new_text)
    # remove punctuation
    new_text = re.sub('[%s]' % re.escape(string.punctuation), '', new_text)

    # Reference : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
    emoji_pattern = re.compile("["
                            u"\U0001F600-\U0001F64F"  # emoticons
                            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                            u"\U0001F680-\U0001F6FF"  # transport & map symbols
                            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                            u"\U00002702-\U000027B0"
                            u"\U000024C2-\U0001F251"
                            "]+", flags=re.UNICODE)
    new_text = emoji_pattern.sub(r'', new_text)
    new_text = re.sub('\n', '', new_text)
    new_text = re.sub('\w*\d\w*', '', new_text)
    new_text = re.sub('\d*', '', new_text)

    stop_words = set(stopwords.words('english'))
    tokens = []
    for word in new_text.split():
        if word not in stop_words:
            tokens.append(lemmatizer.lemmatize(word))
    new_text = ' '.join(tokens)
    return new_text

def extract_emoticons(text):
    '''Extract the last token
    '''
    new_text = clean_text(text, set_lower=False)
    words = new_text.split()
    return words[-1] if words else ''

def count_all_capitalized(text):
    new_text = clean_text(text, set_lower=False)
    return len([word for word in new_text.split() if word.isupper()])
        

In [27]:
df_train['text_clean'] = df_train['text'].apply(clean_text)
df_dev['text_clean'] = df_dev['text'].apply(clean_text)
df_test['text_clean'] = df_test['text'].apply(clean_text)

# df_train['emoticons'] = df_train['text'].apply(extract_emoticons)
# df_dev['emoticons'] = df_dev['text'].apply(extract_emoticons)
# df_test['emoticons'] = df_test['text'].apply(extract_emoticons)

# df_train['num_all_capitalized'] = df_train['text'].apply(count_all_capitalized)
# df_dev['num_all_capitalized'] = df_dev['text'].apply(count_all_capitalized)
# df_test['num_all_capitalized'] = df_test['text'].apply(count_all_capitalized)

In [28]:
# wc = wordcloud.WordCloud(
#     background_color='white', 
#     max_words=200, 
# #     mask=twitter_mask,
# )
# wc.generate(' '.join(text for text in df.loc[df['label'] == 0, 'text_clean']))
# plt.figure(figsize=(18,10))
# plt.title('Top words for NON-TOXIC messages', 
#           fontdict={'size': 22,  'verticalalignment': 'bottom'})
# plt.imshow(wc)
# plt.axis("off")
# plt.show()

In [29]:
# wc = wordcloud.WordCloud(
#     background_color='white', 
#     max_words=200, 
# #     mask=twitter_mask,
# )
# wc.generate(' '.join(text for text in df.loc[df['label'] == 1, 'text_clean']))
# plt.figure(figsize=(18,10))
# plt.title('Top words for TOXIC messages', 
#           fontdict={'size': 22,  'verticalalignment': 'bottom'})
# plt.imshow(wc)
# plt.axis("off")
# plt.show()

In [30]:
df_train

,id,text,label,text_clean
0,0,Except that Desmond played first base last nig...,0,except desmond played first base last night ta...
1,1,What i find funny is the loyalty and blindness...,0,find funny loyalty blindness english community...
2,2,Read the article not just the headline & you ...,0,read article headline find
3,3,Speaking of a horses backside is that where y...,1,speaking horses backside head
4,4,Michael Barone- gee are you dumb. No other wo...,1,michael barone gee dumb words needed
...,...,...,...,...
98995,98995,the libs could just pass a law that pulls them...,1,libs could pass law pulls treaty easily exited...
98996,98996,Really? How does this post in any way relate t...,0,really post way relate article article “we can...
98997,98997,Hey illegals if your reading this get the hel...,1,hey illegals reading get hell country
98998,98998,"Excellent description ""he playground bully ol...",1,excellent description playground bully old lit...


## Preprocessing

In [22]:
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline, make_pipeline

from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
# from sklearn.pipeline import Pipeline, make_pipeline

x_train, y_train = df_train['text_clean'], df_train['label']

preprocessing_pipe = Pipeline([
                    ('bow', CountVectorizer()), 
                    ('tfidf', TfidfTransformer()),
                ])
preprocessing_pipe.fit(x_train)
x_train_tfidf = preprocessing_pipe.transform(x_train)

sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(x_train_tfidf, y_train)

In [18]:
import joblib

# save
joblib.dump(preprocessing_pipe, "tfidf.pkl") 

['tfidf.pkl']

In [39]:
import os

from scipy import sparse

os.makedirs('output', exist_ok=True)

sparse.save_npz("output/x_train_smote.npz", X_res)
sparse.save_npz("output/y_train_smote.npz", sparse.csr_matrix(y_res.values))